In [216]:
#
# This file scrap ft.dk to gather the list of MP in the danish government 
#
# Import packages

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup
import time

from urllib3.exceptions import InsecureRequestWarning

# Header to be used in request (with contact info)
header = {'Contact' : 'Christofferfoldager@gmail.com'}

{'Contact': 'Christofferfoldager@gmail.com', 'Info': 'Gather list of MP'}

In [222]:
import urllib.request
url = 'https://www.ft.dk/da/medlemmer/mandatfordelingen'
# requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)
header['Info'] = 'Gather list of MP for research project'
page = requests.get(url,verify=False, headers=header)
soup = BeautifulSoup(page.text, 'html.parser')

# Get the links
data = soup.find_all('table')[0]
aref = data.find_all('a')
member_nr = data.find_all('div', {'class':'member-container'})

# Create a list with all relevant infomation 
Parties = []

for i in range(len(aref)):
    Parties.append([aref[i].text, aref[i].get('href'),member_nr[i].text])

# Creates a Dataframe    
df_Parties = pd.DataFrame(Parties, columns=['Party','AddressLink','Members'])
df_Parties

,Party,AddressLink,Members
0,Socialdemokratiet (S),/searchResults.aspx?sortedDescending=false&par...,49
1,Venstre (V),/searchResults.aspx?sortedDescending=false&par...,39
2,Dansk Folkeparti (DF),/searchResults.aspx?sortedDescending=false&par...,16
3,Socialistisk Folkeparti (SF),/searchResults.aspx?sortedDescending=false&par...,15
4,Radikale Venstre (RV),/searchResults.aspx?sortedDescending=false&par...,14
5,Enhedslisten (EL),/searchResults.aspx?sortedDescending=false&par...,13
6,Det Konservative Folkeparti (KF),/searchResults.aspx?sortedDescending=false&par...,13
7,Nye Borgerlige (NB),/searchResults.aspx?sortedDescending=false&par...,4
8,Liberal Alliance (LA),/searchResults.aspx?sortedDescending=false&par...,3
9,Alternativet (ALT),/searchResults.aspx?sortedDescending=false&par...,1


In [546]:
ID_list = []

for i in range(len(df_Parties)): # Loop for each party
    Party = df_Parties['Party'][i]

    #Get the relevant party page
    link = df_Parties['AddressLink'][i]
    url = f'http://ft.dk{link}&page=1&sortedBy=&pageSize=50'
    # time.sleep(1)
    page = requests.get(url, verify=False, headers=header)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Get the names 
    tables = soup.find_all('tr',{'tabindex':0})

    for i in range(len(tables)): # Loop for each member of the given party 
        FirstName = tables[i].find_all('td')[1].text
        LastName = tables[i].find_all('a')[0].text
        Name = FirstName + ' ' + LastName
        MP = tables[i].find_all('td')[4].text

        # Create a first, last and middle name
        Name_list = re.findall('\w+',Name)
        First_Name = Name_list[0]
        Last_Name = Name_list[-1]
        Middel_Name = ' '.join(Name_list[1:-1])

        ID_list.append([Name,Party,MP,First_Name,Last_Name,Middel_Name])

df_Name = pd.DataFrame(ID_list, columns=['Name','Party','MP','First Name','Last Name','Middel Name'])

In [224]:
df_Name.to_csv('NameId.csv')
df_Parties.to_csv('Parties.csv')

In [529]:
pd.set_option('display.max_rows', None)
df_Name.head(5)

,Name,Party,MP,First Name,Last Name
0,Ida Auken,Socialdemokratiet (S),Medlem af Folketinget,Ida,Auken
1,Trine Bramsen,Socialdemokratiet (S),Medlem af Folketinget,Trine,Bramsen
2,Bjørn Brandenborg,Socialdemokratiet (S),Medlem af Folketinget,Bjørn,Brandenborg
3,Jeppe Bruus,Socialdemokratiet (S),Medlem af Folketinget,Jeppe,Bruus
4,Morten Bødskov,Socialdemokratiet (S),Medlem af Folketinget,Morten,Bødskov


In [547]:
url = 'https://filip.sdu.dk/twitter/politikere/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

data = soup.find_all('div', {'class':'col-md-6 show_tweet show_user'}) # Finds the list of all politicians twitter account

Twitter_list = []

for i in range(len(data)):
    info_i = data[i].find_all('div', {'class':'media-body'})
    twitter_id = info_i[0].find('h3').text.split()
    person_id = info_i[0].find('small').text.split(' \nmed ')

    Name = person_id[0]
    Tag = twitter_id[1]
    Følgere = re.search('[0-9.]+[0-9]|\d', person_id[1]).group()

    # Create a first, last and middle name
    Name_list = re.findall('\w+',Name)
    First_Name = Name_list[0]
    Last_Name = Name_list[-1]
    Middel_Name = ' '.join(Name_list[1:-1])

    Twitter_list.append([Name,Tag,Følgere, First_Name, Last_Name,Middel_Name])

info_i


df_Twitter_id = pd.DataFrame(Twitter_list, columns=['Name','Twitter_id','Følgere','First Name','Last Name','Middel Name'])
df_Twitter_id
# info_i = data[18].find_all('div', {'class':'media-body'})
# info_i

,Name,Twitter_id,Følgere,First Name,Last Name,Middel Name
0,Margrethe Vestager,@vestager,295.327,Margrethe,Vestager,
1,Lars Løkke Rasmussen,@larsloekke,198.345,Lars,Rasmussen,Løkke
2,Pernille Skipper,@PSkipperEL,82.871,Pernille,Skipper,
3,Ida Auken,@IdaAuken,73.926,Ida,Auken,
4,Kristian Jensen,@Kristian_Jensen,62.946,Kristian,Jensen,
5,Søren Pape Poulsen,@SorenPape,61.615,Søren,Poulsen,Pape
6,Magnus Heunicke,@Heunicke,60.605,Magnus,Heunicke,
7,Uffe Elbaek,@uffeelbaek,58.523,Uffe,Elbaek,
8,Statsministeriet,@Statsmin,58.426,Statsministeriet,Statsministeriet,
9,Pia Olsen Dyhr,@PiaOlsen,56.368,Pia,Dyhr,Olsen


In [525]:
N = 'Lars Løkke Fizz Rasmussen'
First_N = re.search('\w+',N)
Second_N = re.findall('\w+',N)
Second_N[1:-1]
A = ' '.join(Second_N[1:-1])
A

'Løkke Fizz'

In [705]:


A_df = df_Name.copy()
B_df = df_Twitter_id.copy()

new_df = pd.merge(A_df, B_df[['First Name', 'Last Name','Twitter_id','Følgere']],  how='left', on=['First Name','Last Name'])
a = pd.merge(A_df, B_df[['First Name', 'Last Name','Twitter_id','Følgere']],  how='left', left_on = ['First Name','Middel Name'], right_on=['First Name', 'Last Name'])

new_df['Twitter_id'] = new_df['Twitter_id'].fillna(a['Twitter_id'])
new_df['Følgere'] = new_df['Følgere'].fillna(a['Følgere'])
new_df


,Name,Party,MP,First Name,Last Name,Middel Name,Twitter_id,Følgere
0,Ida Auken,Socialdemokratiet (S),Medlem af Folketinget,Ida,Auken,,@IdaAuken,73.926
1,Trine Bramsen,Socialdemokratiet (S),Medlem af Folketinget,Trine,Bramsen,,@Trinebramsen,13.483
2,Bjørn Brandenborg,Socialdemokratiet (S),Medlem af Folketinget,Bjørn,Brandenborg,,@BjBrandenborg,2.277
3,Jeppe Bruus,Socialdemokratiet (S),Medlem af Folketinget,Jeppe,Bruus,,@JeppeBruus,4.066
4,Morten Bødskov,Socialdemokratiet (S),Medlem af Folketinget,Morten,Bødskov,,@mfMorten,15.324
5,Lennart Damsbo-Andersen,Socialdemokratiet (S),Medlem af Folketinget,Lennart,Andersen,Damsbo,NaN,NaN
6,Kaare Dybvad Bek,Socialdemokratiet (S),Medlem af Folketinget,Kaare,Bek,Dybvad,@KaareDybvad,7.432
7,Benny Engelbrecht,Socialdemokratiet (S),Medlem af Folketinget,Benny,Engelbrecht,,@BennyEngelbrech,20.829
8,Camilla Fabricius,Socialdemokratiet (S),Medlem af Folketinget,Camilla,Fabricius,,@CamFabricius,2.147
9,Mette Frederiksen,Socialdemokratiet (S),Medlem af Folketinget,Mette,Frederiksen,,NaN,NaN


In [703]:
# Merge the Dataframes 
# df_info = df_Name.merge(df_Twitter_id,how='left')
df_info = new_df.copy()
df_info.replace(float('NaN'),'None',inplace=True)

# Create two new dataframes - the ones with (without) twitter
df_Twitter_missing = df_info.loc[df_info['Twitter_id'] == 'None']
df_info_list = df_info.loc[df_info['Twitter_id'] != 'None']


# Kaare Dybvad Bek

df_info_list
Red = ['Socialdemokratiet (S)','Socialistisk Folkeparti (SF)','Radikale Venstre (RV)','Enhedslisten (EL)','Alternativet (ALT)']
Names_Ind = ['Susanne Zimmer']
# df_new = df_info_list[df_info_list['Party'].isin(Red) = 1]
# df_info_list
df_info_list['y'] = np.where(df_info_list['Party'].isin(Red),'Red','Blue')
df_info_list


<ipython-input-703-a1a1791b9e1e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_info_list['y'] = np.where(df_info_list['Party'].isin(Red),'Red','Blue')


,Name,Party,MP,First Name,Last Name,Middel Name,Twitter_id,Følgere,Føgere,y
0,Ida Auken,Socialdemokratiet (S),Medlem af Folketinget,Ida,Auken,,@IdaAuken,73.926,73.926,Red
1,Trine Bramsen,Socialdemokratiet (S),Medlem af Folketinget,Trine,Bramsen,,@Trinebramsen,13.483,13.483,Red
2,Bjørn Brandenborg,Socialdemokratiet (S),Medlem af Folketinget,Bjørn,Brandenborg,,@BjBrandenborg,2.277,2.277,Red
3,Jeppe Bruus,Socialdemokratiet (S),Medlem af Folketinget,Jeppe,Bruus,,@JeppeBruus,4.066,4.066,Red
4,Morten Bødskov,Socialdemokratiet (S),Medlem af Folketinget,Morten,Bødskov,,@mfMorten,15.324,15.324,Red
6,Kaare Dybvad Bek,Socialdemokratiet (S),Medlem af Folketinget,Kaare,Bek,Dybvad,@KaareDybvad,None,7.432,Red
7,Benny Engelbrecht,Socialdemokratiet (S),Medlem af Folketinget,Benny,Engelbrecht,,@BennyEngelbrech,20.829,20.829,Red
8,Camilla Fabricius,Socialdemokratiet (S),Medlem af Folketinget,Camilla,Fabricius,,@CamFabricius,2.147,2.147,Red
10,Mette Gjerskov,Socialdemokratiet (S),Medlem af Folketinget,Mette,Gjerskov,,@MetteGjerskov,12.388,12.388,Red
12,Ane Halsboe-Jørgensen,Socialdemokratiet (S),Medlem af Folketinget,Ane,Jørgensen,Halsboe,@AneHalsboe,None,10.410,Red
